In [1]:
import json
import spacy
import scispacy
from scispacy.umls_linking import UmlsEntityLinker

In [2]:
nlp = spacy.load("en_ner_bc5cdr_md")
text = ("Here is the caveat........... Tony will always have issues with his brain. He used to constantly worry about things that have no significance. For example, he would tell me that he is worrying about something that happened 8 months ago or worry that he has upset or possibly offended me, a teacher or a neighbor in some way. His worrying used to dictate his life after the Singulair. I'm explaining this because I want everyone to know what has helped Tony and not only changed his life but our family's life. After MUCH research, I put Tony on a protocol of vitamins. The MOST important being 1000 mg. of Niacin 3x's a day (3 grams total per day) - NON FLUSHING!!!! The 2nd most important supplement I give him is St. John's Wort. If we are in a hurry, he MUST take at the very least those two supplements and when we get back home he can take the rest. We have been in the car on our way to dinner and he has asked me to turn around because he forgot the Niacin - it has changed his life. I now ALWAYS keep an extra bottle Niacin in the car.")
doc = nlp(text)

In [3]:
with open("../additional resources/drug_disease.json", "r") as file:
    drug_disease = json.load(file)

In [4]:
def get_disease(drug):
    try:
        try:
            return sorted(drug_disease[drug])
        except:
            drug = drug.capitalize()
            return sorted(drug_disease[drug])
    except:
        return "Drug Not found!!!"

In [5]:
drugs = []
for entity in doc.ents:
    drug = entity.text
    label = entity.label_
    if label=="CHEMICAL" and drug not in drugs:
        print(get_disease(drug))
    elif label=="DISEASE":
        print(drug)
    drugs.append(drug)

Drug Not found!!!
['Acidosis', 'Acidosis, Lactic', 'Alzheimer Disease', 'Amblyopia', 'Anemia, Megaloblastic', 'Arteriosclerosis', 'Atherosclerosis', 'Breast Neoplasms', 'Cardiovascular Diseases', 'Carotid Artery Diseases', 'Cataract', 'Cattle Diseases', 'Chemical and Drug Induced Liver Injury', 'Chills', 'Cholestasis', 'Cholestasis, Intrahepatic', 'Chromosomal Instability', 'Cleft Lip', 'Cleft Palate', 'Cognition Disorders', 'Coronary Artery Disease', 'Coronary Disease', 'Coronary Stenosis', 'Delirium', 'Diabetes Mellitus', 'Diabetic Angiopathies', 'Dizziness', 'Drug-Related Side Effects and Adverse Reactions', 'Dyskinesias', 'Dyslipidemias', 'Dyspepsia', 'Edema', 'Endotoxemia', 'Erythema', 'Exanthema', 'Exophthalmos', 'Eye Diseases', 'Eyelid Diseases', 'Fatty Liver', 'Flushing', 'Gastrointestinal Diseases', 'Glomerulosclerosis, Focal Segmental', 'Glucose Intolerance', 'Gout', 'Hepatic Encephalopathy', 'Hepatitis', 'Hypercholesterolemia', 'Hyperglycemia', 'Hyperhomocysteinemia', 'Hyper